In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [4]:
import os

os.listdir("../data")


['CEAS_08.csv', 'malicious_phish.csv', 'spam.csv']

In [5]:
import pandas as pd

df1 = pd.read_csv("../data/CEAS_08.csv", encoding='latin1')
df2 = pd.read_csv("../data/malicious_phish.csv", encoding='latin1')
df3 = pd.read_csv("../data/spam.csv", encoding='latin1')

df1.head(), df2.head(), df3.head()


(                                              sender  \
 0                   Young Esposito <Young@iworld.de>   
 1                       Mok <ipline's1983@icable.ph>   
 2  Daily Top 10 <Karmandeep-opengevl@universalnet...   
 3                 Michael Parker <ivqrnai@pobox.com>   
 4  Gretchen Suggs <externalsep1@loanofficertool.com>   
 
                                          receiver  \
 0                     user4@gvc.ceas-challenge.cc   
 1                   user2.2@gvc.ceas-challenge.cc   
 2                   user2.9@gvc.ceas-challenge.cc   
 3  SpamAssassin Dev <xrh@spamassassin.apache.org>   
 4                   user2.2@gvc.ceas-challenge.cc   
 
                               date  \
 0  Tue, 05 Aug 2008 16:31:02 -0700   
 1  Tue, 05 Aug 2008 18:31:03 -0500   
 2  Tue, 05 Aug 2008 20:28:00 -1200   
 3  Tue, 05 Aug 2008 17:31:20 -0600   
 4  Tue, 05 Aug 2008 19:31:21 -0400   
 
                                              subject  \
 0                          Never agr

In [6]:
df1.info()
df2.info()
df3.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39154 entries, 0 to 39153
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    39154 non-null  object
 1   receiver  38692 non-null  object
 2   date      39154 non-null  object
 3   subject   39126 non-null  object
 4   body      39154 non-null  object
 5   label     39154 non-null  int64 
 6   urls      39154 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 2.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1         

In [7]:
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [8]:
df = df2.copy()

df['label'] = df['type'].apply(lambda x: 1 if x != 'benign' else 0)

df.head()
df['label'].value_counts()


label
0    428103
1    223088
Name: count, dtype: int64

In [9]:
df = df[['url', 'label']]
df.head()


,url,label
0,br-icloud.com.br,1
1,mp3raid.com/music/krizz_kaliko.html,0
2,bopsecrets.org/rexroth/cr/1.htm,0
3,http://www.garage-pirenne.be/index.php?option=...,1
4,http://adventure-nicaragua.net/index.php?optio...,1


In [10]:
import re

def extract_features(url):
    features = {}
    features['url_length'] = len(url)
    features['num_dots'] = url.count('.')
    features['num_slashes'] = url.count('/')
    features['has_https'] = 1 if "https" in url else 0
    features['has_ip'] = 1 if re.match(r'\d+\.\d+\.\d+\.\d+', url) else 0
    features['num_special_chars'] = len(re.findall(r'[@!?&%$#=]', url))
    return features

feature_data = df['url'].apply(extract_features)
feature_df = pd.DataFrame(feature_data.tolist())

final_df = pd.concat([feature_df, df['label']], axis=1)
final_df.head()


,url_length,num_dots,num_slashes,has_https,has_ip,num_special_chars,label
0,16,2,0,0,0,0,1
1,35,2,2,0,0,0,0
2,31,2,3,0,0,0,0
3,88,3,3,0,0,8,1
4,235,2,3,0,0,6,1


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = final_df.drop('label', axis=1)
y = final_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.8799745084037808

In [12]:
import joblib

joblib.dump(model, "../model/phishing_model.pkl")


FileNotFoundError: [Errno 2] No such file or directory: '../model/phishing_model.pkl'

In [13]:
joblib.dump(model, "../model/phishing_model.pkl") 

FileNotFoundError: [Errno 2] No such file or directory: '../model/phishing_model.pkl'

In [14]:
joblib.dump(model, "phishing_model.pkl")
print("Model saved successfully!")


Model saved successfully!
